#### Import packages and data

In [163]:
import pandas as pd
import numpy as np
import seaborn as sns
import time

from glob import glob
import matplotlib.pyplot as plt
%matplotlib inline

from scipy import stats 

In [164]:
business=pd.read_csv('C:/Users/Ric/Desktop/000_MY_WORKDIRECTORY/python/capstone yelp/yelp_business_useV3.csv')
demographics=pd.read_csv('C:/Users/Ric/Desktop/000_MY_WORKDIRECTORY/python/capstone yelp/Census.csv')

In [165]:
business.head(2)

,business_id,name,neighborhood,address,city,state,postal_code,latitude,longitude,stars,review_count,is_open,categories
0,__aKnGBedQ51_hEc3D9ARw,"""Paradise Bakery & Cafe""",NaN,"""2502 E Camelback Rd, Ste 119""",Phoenix,AZ,85016,33.510270,-112.028937,3.0,75,0,Sandwiches;Restaurants;Food;Bakeries;Juice Bar...
1,_4bbY5joxS7ZM-027Gf6Jw,"""Mikado Sushi""",NaN,"""3941 E Chandler Blvd, Ste 107""",Phoenix,AZ,85048,33.305108,-111.999400,4.5,7,0,Restaurants;Japanese


#### Generate new column for single category

In [166]:
business['cat'] = "NA"

#### Generate Dummy Variable Columns

In [167]:
list_dv = ['Fast Food','Bars','Gluten-Free','Vegetarian','Vegan','Pizza']

In [168]:
for i in list_dv: 
    business[i] = business['categories'].str.contains(i, regex=False)

In [169]:
business.head(2)

,business_id,name,neighborhood,address,city,state,postal_code,latitude,longitude,stars,review_count,is_open,categories,cat,Fast Food,Bars,Gluten-Free,Vegetarian,Vegan,Pizza
0,__aKnGBedQ51_hEc3D9ARw,"""Paradise Bakery & Cafe""",NaN,"""2502 E Camelback Rd, Ste 119""",Phoenix,AZ,85016,33.510270,-112.028937,3.0,75,0,Sandwiches;Restaurants;Food;Bakeries;Juice Bar...,NA,False,True,False,False,False,False
1,_4bbY5joxS7ZM-027Gf6Jw,"""Mikado Sushi""",NaN,"""3941 E Chandler Blvd, Ste 107""",Phoenix,AZ,85048,33.305108,-111.999400,4.5,7,0,Restaurants;Japanese,NA,False,False,False,False,False,False


#### Filter by restaurants

In [170]:
business['temp'] = "0"
business['temp'] = business['categories'].str.contains('Restaurant', regex=False)
business = business[business.temp == True]
business['temp'].value_counts()

True    3568
Name: temp, dtype: int64

#### Categorize a restaurant based on categories

In [171]:
list_cat = ['Nightlife','Music Venues','Venues & Event Spaces','Arts & Entertainment','Grocery',
            
            #######
            
            #generic categories go first
            'Comfort Food',
            'Bars',
            'Fast Food',
            'Breakfast & Brunch',
            'Desserts',           
            'Cafes','Bakeries','Delis','Cafeteria',
            'American (Traditional)',
            'American (New)',
            'Seafood',
            'Buffets',
            'Gluten-Free','Vegetarian', 'Vegan',
            'Cheesesteaks','Chicken',
            'Hot Dogs',
            
            #Filter through American food
            'Sandwiches','Burgers','Sports Bars','Barbeque','Steakhouses',
            #Filter through central American food
            'Latin America','Mexican','Tex-Mex','Peruvian','Salvadoran',
            #Filter through Mediterranean food
            'Mediterranean','Greek',
            #Filter through Asian Food
            'Chinese','Asian Fusion','Thai','Vietnamese','Japanese','Sushi',
            'Korean',
            #Filter through Middle East
            'Middle Eastern',
            
            #Filter through pizza and Italian
            'Pizza','Italian',
            #Filter through Carribean
            'Carribean','Caribbean','Cajun','Cuban',
            #Filter through European 
            'Modern European','French',
            'Russian','Ukranian','Polish','Uzbek','Afghan',
            'German','Bavarian',
            'Spanish',
            #Filter through African
            'African','Ethiopian',
            #Filter throuh Indian
            'Indian',#'Pakistani',
                     
            #Filter through Southern
            'Southern','Diners','Soul Food',
            #Filter through bar restaurants
            'Sports Bar','Wine Bars','Gastropub','Cocktail Bars','Dive Bar',
            #Filter through misc.
            'Coffee & Tea','Juice Bars & Smoothies','Ice Cream & Frozen Yogurt',
            'Bagels', 'Donuts',
            'Fish & Chips','Kosher','Pretzels',
            'Hawaiian', 
            
            ########
            
            #Label things that are not restaurants
            
            'Festival','Food Trucks','Event Planning & Services','Grocery', 
            'Food Delivery Services','Shopping','Active Life', 'Party & Event Planning',
            'Health Markets','Convenience Stores', 'Arcades','Professional Services',
            'Home & Garden','Dance Clubs', 'Wholesalers','Restaurant Supplies',
            'Automotive','Furniture Stores', 'Health & Medical','Gas Stations',
            'Kitchen & Bath','Beauty & Spas', 'Home Services','Appliances','Day Spas',
            'Personal Chefs','Caterers', 'Fashion','Food Court','Veterinarians'
            
             ]

In [172]:
for i in list_cat: 
    business['temp'] = "0"
    business['temp'] = business['categories'].str.contains(i, regex=False)
    business['cat'] = np.where(business['temp'] == True, i, business['cat'])

In [173]:
business['cat'].value_counts()

Mexican                      498
Sandwiches                   261
Burgers                      260
Italian                      217
Pizza                        188
Chinese                      161
American (Traditional)       115
Caterers                     110
American (New)                93
Coffee & Tea                  92
Sports Bar                    91
Sushi                         72
Tex-Mex                       61
Diners                        59
Greek                         58
Fast Food                     53
Food Trucks                   51
Barbeque                      51
Wine Bars                     49
Ice Cream & Frozen Yogurt     45
Chicken                       44
Cocktail Bars                 41
Steakhouses                   40
Thai                          40
Vietnamese                    37
Indian                        33
Juice Bars & Smoothies        33
Asian Fusion                  32
Japanese                      31
Grocery                       27
Bagels    

#### Remove non-restaurant categories that still remain

In [174]:
list_drop = ['Appliances','Home Services','Festival','Health & Medical',
             'Gas Stations','Day Spas','Restaurant Supplies',
             'Personal Chefs','Veterinarians','Arts & Entertainment',
             'Kitchen & Bath','Nightlife','Furniture Stores','Beauty & Spas',
             'Arcades','Dance Clubs','Party & Event Planning',
             'Convenience Stores','Caterers','Food Trucks','Active Life',
             'Food Delivery Services','Health Markets','Shopping','Grocery',
             'Event Planning & Services','Fashion'
            ]

In [175]:
for i in list_drop:
    business = business.loc[business['cat'] != i]

In [176]:
business.shape[0]

3247

In [177]:
pd.set_option('display.max_row', 100)
business['cat'].value_counts()

Mexican                      498
Sandwiches                   261
Burgers                      260
Italian                      217
Pizza                        188
Chinese                      161
American (Traditional)       115
American (New)                93
Coffee & Tea                  92
Sports Bar                    91
Sushi                         72
Tex-Mex                       61
Diners                        59
Greek                         58
Fast Food                     53
Barbeque                      51
Wine Bars                     49
Ice Cream & Frozen Yogurt     45
Chicken                       44
Cocktail Bars                 41
Thai                          40
Steakhouses                   40
Vietnamese                    37
Indian                        33
Juice Bars & Smoothies        33
Asian Fusion                  32
Japanese                      31
Bagels                        27
Middle Eastern                26
Mediterranean                 25
NA        

#### Categorize a restaurant by its name

In [178]:
business['temp'] = business['name'].str.contains('Burger King', regex=False)
business['cat'] = np.where(business['temp'] == True, 'Burger', business['cat'])

In [179]:
business['temp'] = business['name'].str.contains('McDonalds', regex=False)
business['cat'] = np.where(business['temp'] == True, 'Burger', business['cat'])

In [180]:
business['temp'] = business['name'].str.contains('Dairy Queen', regex=False)
business['cat'] = np.where(business['temp'] == True, 'Ice Cream & Frozen Yogurt', business['cat'])

In [181]:
business['temp'] = business['name'].str.contains('Noodles & Company', regex=False)
business['cat'] = np.where(business['temp'] == True, 'American (New)', business['cat'])

In [182]:
business['temp'] = business['name'].str.contains('The Loaded Potato', regex=False)
business['cat'] = np.where(business['temp'] == True, 'American (New)', business['cat'])

In [183]:
business['temp'] = business['name'].str.contains('Food Court', regex=False)
business['cat'] = np.where(business['temp'] == True, 'Food Court', business['cat'])

In [184]:
business['temp'] = business['name'].str.contains('Sonic', regex=False)
business['cat'] = np.where(business['temp'] == True, 'Ice Cream & Frozen Yogurt', business['cat'])

In [185]:
business['temp'] = business['name'].str.contains('Mighty Mikes', regex=False)
business['cat'] = np.where(business['temp'] == True, 'Burger', business['cat'])

In [186]:
business['temp'] = business['name'].str.contains('Salad', regex=False)
business['cat'] = np.where(business['temp'] == True, 'Salad', business['cat'])

In [187]:
business['temp'] = business['name'].str.contains('Pizza', regex=False)
business['cat'] = np.where(business['temp'] == True, 'Pizza', business['cat'])

In [188]:
business['temp'] = business['name'].str.contains('pizza', regex=False)
business['cat'] = np.where(business['temp'] == True, 'pizza', business['cat'])

In [189]:
business['temp'] = business['name'].str.contains('Taco', regex=False)
business['cat'] = np.where(business['temp'] == True, 'Mexican', business['cat'])

In [190]:
business['temp'] = business['name'].str.contains('taco', regex=False)
business['cat'] = np.where(business['temp'] == True, 'Mexican', business['cat'])

In [191]:
business['temp'] = business['name'].str.contains('Carl', regex=False)
business['cat'] = np.where(business['temp'] == True, 'Burgers', business['cat'])

In [192]:
business['temp'] = business['name'].str.contains('Red Robin', regex=False)
business['cat'] = np.where(business['temp'] == True, 'Burgers', business['cat'])

In [193]:
business['temp'] = business['name'].str.contains(r'(?=.*Jack)(?=.*Box)',regex=True)
business['cat'] = np.where(business['temp'] == True, 'Burgers', business['cat'])

In [194]:
business['temp'] = business['name'].str.contains(r'(?=.*Jack)(?=.*box)',regex=True)
business['cat'] = np.where(business['temp'] == True, 'Burgers', business['cat'])

In [195]:
business['temp'] = business['name'].str.contains('Chicken', regex=False)
business['cat'] = np.where(business['temp'] == True, 'Chicken', business['cat'])

In [196]:
business['temp'] = business['name'].str.contains('Fish', regex=False)
business['cat'] = np.where(business['temp'] == True, 'Seafood', business['cat'])

In [197]:
business['temp'] = business['name'].str.contains('Fish & Chips', regex=False)
business['cat'] = np.where(business['temp'] == True, 'Fish & Chips', business['cat'])

In [198]:
business['temp'] = business['name'].str.contains('Wong', regex=False)
business['cat'] = np.where(business['temp'] == True, 'Chinese', business['cat'])

In [199]:
business['temp'] = business['name'].str.contains('Grill', regex=False)
business['cat'] = np.where(business['temp'] == True, 'American (New)', business['cat'])

In [200]:
business['temp'] = business['name'].str.contains('Cafe', regex=False)
business['cat'] = np.where(business['temp'] == True, 'Cafe', business['cat'])

In [201]:
business['temp'] = business['name'].str.contains('Deli', regex=False)
business['cat'] = np.where(business['temp'] == True, 'Deli', business['cat'])

In [202]:
business['temp'] = business['name'].str.contains('Pretzels', regex=False)
business['cat'] = np.where(business['temp'] == True, 'Pretzels', business['cat'])

In [203]:
business['temp'] = business['name'].str.contains('Nueva', regex=False)
business['cat'] = np.where(business['temp'] == True, 'Mexican', business['cat'])

In [204]:
business['temp'] = business['name'].str.contains('Pueblo', regex=False)
business['cat'] = np.where(business['temp'] == True, 'Mexican', business['cat'])

In [205]:
business['temp'] = business['name'].str.contains('Salsita', regex=False)
business['cat'] = np.where(business['temp'] == True, 'Mexican', business['cat'])

In [206]:
pd.set_option('display.max_row', 100)
business['cat'].value_counts()

Mexican                      478
American (New)               317
Pizza                        272
Burgers                      255
Sandwiches                   220
Chinese                      165
Cafe                         163
Italian                      120
Coffee & Tea                  71
Sushi                         65
Chicken                       60
American (Traditional)        59
Ice Cream & Frozen Yogurt     57
Diners                        50
Barbeque                      48
Greek                         47
Sports Bar                    47
Burger                        40
Wine Bars                     39
Thai                          39
Cocktail Bars                 39
Deli                          37
Steakhouses                   37
Vietnamese                    36
Indian                        28
Japanese                      27
Juice Bars & Smoothies        26
Asian Fusion                  26
Seafood                       22
Bagels                        20
Mediterran

#### Consolidate categories

In [207]:
business['temp'] = business['cat'].str.contains('Bavarian', regex=False)
business['cat'] = np.where(business['temp'] == True, 'German', business['cat'])

In [208]:
business['temp'] = business['cat'].str.contains('Salvadoran', regex=False)
business['cat'] = np.where(business['temp'] == True, 'Latin America', business['cat'])

In [209]:
business['temp'] = business['cat'].str.contains('Peruvian', regex=False)
business['cat'] = np.where(business['temp'] == True, 'Latin America', business['cat'])

In [210]:
business['temp'] = business['cat'].str.contains('Ethiopian', regex=False)
business['cat'] = np.where(business['temp'] == True, 'African', business['cat'])

In [211]:
business['temp'] = business['cat'].str.contains('Cheesesteaks', regex=False)
business['cat'] = np.where(business['temp'] == True, 'Sandwiches', business['cat'])

In [212]:
business['temp'] = business['cat'].str.contains('pizza', regex=False)
business['cat'] = np.where(business['temp'] == True, 'Pizza', business['cat'])

In [213]:
business['cat'].value_counts()

Mexican                      478
American (New)               317
Pizza                        274
Burgers                      255
Sandwiches                   221
Chinese                      165
Cafe                         163
Italian                      120
Coffee & Tea                  71
Sushi                         65
Chicken                       60
American (Traditional)        59
Ice Cream & Frozen Yogurt     57
Diners                        50
Barbeque                      48
Greek                         47
Sports Bar                    47
Burger                        40
Thai                          39
Cocktail Bars                 39
Wine Bars                     39
Deli                          37
Steakhouses                   37
Vietnamese                    36
Indian                        28
Japanese                      27
Juice Bars & Smoothies        26
Asian Fusion                  26
Seafood                       22
Bagels                        20
Mediterran

#### National chains (https://www.restaurantbusinessonline.com/top-500-chains)

In [214]:
business['chain'] = False
business['chain'] = business.groupby(['name']).transform('count')

In [215]:
business['chain'] = np.where(business['chain'] > 1, True, False)

In [216]:
business['Fast Food'].value_counts()

False    2682
True      565
Name: Fast Food, dtype: int64

In [217]:
business['Bars'].value_counts()

False    2740
True      507
Name: Bars, dtype: int64

In [218]:
business['Gluten-Free'].value_counts()

False    3199
True       48
Name: Gluten-Free, dtype: int64

In [219]:
business['Vegetarian'].value_counts()

False    3183
True       64
Name: Vegetarian, dtype: int64

In [220]:
business['Vegan'].value_counts()

False    3213
True       34
Name: Vegan, dtype: int64

In [221]:
business['Pizza'].value_counts()

False    2869
True      378
Name: Pizza, dtype: int64

In [222]:
business['chain'].value_counts()

False    2048
True     1199
Name: chain, dtype: int64

In [223]:
business.head(20)
business.to_csv('C:/Users/Ric/Desktop/000_MY_WORKDIRECTORY/python/capstone yelp/business.csv',sep=',',index= False)

#### Use the code below to interrogate categories

In [224]:
i = False #write in what category you are interested in

In [225]:
bizzy = business.loc[business['chain']==i]
pd.set_option('display.max_row', 100)
bizzy['name'].value_counts() ==1

"JJ's Grill"                                   True
"Sylvia's La Canasta"                          True
"Maya Mexican Antonjitos Snacks"               True
"Castaways"                                    True
"Mi Comida Restaurante Latino"                 True
"Red Devil Italian Restaurant & Pizzeria"      True
"Torta Rica"                                   True
"Caffe Daniel"                                 True
"Gateway Cafe"                                 True
"Sir Veza's Taco Garage"                       True
"Best of Philly"                               True
"Kneaders"                                     True
"Birrieria Las Marias"                         True
"Vovomeena"                                    True
"Petes Fish & Chips"                           True
"Coyote Grill"                                 True
"Mad Chef Gastropub"                           True
"Stacy's Pampered Pig"                         True
"Melt Gelato and Crepe Cafe"                   True
"Sushi Mocor

In [226]:
bizzy.head(20)

,business_id,name,neighborhood,address,city,state,postal_code,latitude,longitude,stars,...,categories,cat,Fast Food,Bars,Gluten-Free,Vegetarian,Vegan,Pizza,temp,chain
1,_4bbY5joxS7ZM-027Gf6Jw,"""Mikado Sushi""",NaN,"""3941 E Chandler Blvd, Ste 107""",Phoenix,AZ,85048,33.305108,-111.999400,4.5,...,Restaurants;Japanese,Japanese,False,False,False,False,False,False,False,False
3,_6mvQX9x66oz046eS5ii8w,"""Juan's Authentic Mexican Food""",NaN,"""1516 E Thomas Rd""",Phoenix,AZ,85014,33.480492,-112.049023,4.0,...,Restaurants;Mexican,Mexican,False,False,False,False,False,False,False,False
4,_70P6YlJVWHQq__6NpIWkA,"""Perc Up Coffee Cafe""",NaN,"""2325 N 7th St""",Phoenix,AZ,85006,33.474169,-112.064757,4.0,...,Cafes;Food;Restaurants;Coffee & Tea,Cafe,False,False,False,False,False,False,False,False
5,_7hhpjQHxjPi0gNzzMmO6g,"""New Sun Star Super Buffet""",NaN,"""5819 S Central Ave""",Phoenix,AZ,85040,33.393691,-112.073216,2.0,...,Buffets;Restaurants;Chinese,Chinese,False,False,False,False,False,False,False,False
8,_8c3Ky90nE50rN4Mb_Sxtw,"""Hsin Cafe""",NaN,"""120 E Taylor St, Ste 150""",Phoenix,AZ,85004,33.453806,-112.071913,3.5,...,Asian Fusion;Restaurants;Chinese,Cafe,False,False,False,False,False,False,False,False
11,_BCwZIK6GYht9cM-HGLduQ,"""Las Jicaras Mexican Grill""",NaN,"""722 W Hatcher Rd""",Phoenix,AZ,85021,33.571532,-112.083540,4.5,...,Restaurants;Mexican,American (New),False,False,False,False,False,False,False,False
12,_bpNAXZqG28e8XplS1wbpw,"""Central Taco""",NaN,"""3110 N Central Ave.""",Phoenix,AZ,85012,33.484142,-112.074694,3.0,...,Mexican;Restaurants,Mexican,False,False,False,False,False,False,False,False
13,_c3ixq9jYKxhLUB0czi0ug,"""Original Hamburger Works""",NaN,"""2801 N 15th Ave""",Phoenix,AZ,85007,33.479789,-112.090994,4.0,...,Nightlife;Sports Bars;Burgers;Dive Bars;Restau...,Dive Bar,False,True,False,False,False,False,False,False
17,_DPJ_PHJnXsuhYhsgBA0Xw,"""Durant's""",NaN,"""2611 N Central Ave""",Phoenix,AZ,85004,33.477206,-112.073533,4.0,...,Steakhouses;Diners;Bars;Wine Bars;Burgers;Seaf...,Wine Bars,False,True,False,False,False,False,False,False
18,_e3rChyednoh47n2cjGv2w,"""Adela's Italian""",NaN,"""4041 E Thomas Rd, Ste 107""",Phoenix,AZ,85018,33.479852,-111.993948,4.5,...,Restaurants;Salad;Italian,Italian,False,False,False,False,False,False,False,False


In [227]:
business_attributes=pd.read_csv('C:/Users/Ric/Desktop/000_MY_WORKDIRECTORY/python/capstone yelp/yelp_business_attributes.csv')
business_hours=pd.read_csv('C:/Users/Ric/Desktop/000_MY_WORKDIRECTORY/python/capstone yelp/yelp_business_hours.csv')

In [228]:
def basic_details(df):
    print('Row:{}, columns:{}'.format(df.shape[0],df.shape[1]))
    k = pd.DataFrame()
    k['number of Unique value'] = df.nunique()
    k['Number of missing value'] = df.isnull().sum()
    k['Data type'] = df.dtypes
    return k

In [229]:
cols_v = list(business_hours.columns.values)[1:]

for i in range(len(cols_v)):
    #print(cols_v[i])
    business_hours[cols_v[i]].replace('None', np.nan, inplace=True)

#business_hours.nunique()

In [230]:
#basic_details(business_hours)

In [231]:




## function for get time_range from string
def get_time_range(s):
    if isinstance(s, str):
        t1, t2 = s.split('-')
        h1, m1 = map(int, t1.split(':'))
        h2, m2 = map(int, t2.split(':'))
        m1, m2 = m1/60, m2/60
        t1, t2 = h1+m1, h2+m2
        if t2 < t1:
            d = t2+24-t1
        else:
            d = t2-t1
        return t1, t2, d
    else:
        return None, None, None

## Prepare start/finish/delta features for every weekday
bh_colnames = business_hours.columns
for c in bh_colnames[1:]:
    business_hours['{0}_s'.format(c[:2])] = business_hours[c].apply(lambda d: get_time_range(d)[0])
    business_hours['{0}_f'.format(c[:2])] = business_hours[c].apply(lambda d: get_time_range(d)[1])
    business_hours['{0}_d'.format(c[:2])] = business_hours[c].apply(lambda d: get_time_range(d)[2])
# business_hours = business_hours.drop(bh_colnames[1:], axis=1)
business_hours


# # define weekday vs weekend categories
# weekdays = ['mo', 'tu', 'we', 'th']
# fridays = ['fr']
# weekends = ['sa', 'su']

# ## define new_cols
# bh_newcols = ['business_id']
# for wg_name, wg in zip(['weekdays', 'fridays', 'weekends'], [weekdays, fridays, weekends]):
#     for f in ['s', 'f', 'd']:
#         cols = list(map(lambda d: '{0}_{1}'.format(d,f), wg))
#         bh_newcols.append('{0}_{1}'.format(wg_name, f))
#         b_hours['{0}_{1}'.format(wg_name, f)] = b_hours.loc[:, cols].median(axis=1)

# b_hours.loc[:, bh_newcols].head(30)

business_hours['monday']=np.where(business_hours.monday.notnull(),1,0)
business_hours['tuesday']=np.where(business_hours.tuesday.notnull(),1,0)
business_hours['wednesday']=np.where(business_hours.wednesday.notnull(),1,0)
business_hours['thursday']=np.where(business_hours.thursday.notnull(),1,0)
business_hours['friday']=np.where(business_hours.friday.notnull(),1,0)
business_hours['saturday']=np.where(business_hours.saturday.notnull(),1,0)
business_hours['sunday']=np.where(business_hours.sunday.notnull(),1,0)



business_hours['open_weekdays'] = np.where((business_hours['monday']==1) & 
                                           (business_hours['tuesday']==1) & 
                                           (business_hours['wednesday']==1) & 
                                           (business_hours['thursday']==1),1,0)
business_hours['open_fridays']=np.where(business_hours['friday']==1,1,0)
business_hours['open_weekends']=np.where((business_hours['saturday']==1)|
                                         (business_hours['sunday']==1),1,0)

business_hours

#break categories down further
#breakfast: 5 a.m. to 10:30 a.m.
#lunch: 11 a.m. to 3:30 p.m.
#dinner: 4 p.m. to 9:30 p.m.
#late_night: 10 p.m. to 4:30 a.m.

business_hours['weekday_breakfast']=np.where(
    (business_hours['mo_s']> 4.5)&(business_hours['mo_s']<10.5)|
    (business_hours['tu_s']> 4.5)&(business_hours['tu_s']<10.5)|
    (business_hours['we_s']> 4.5)&(business_hours['we_s']<10.5)|
    (business_hours['th_s']> 4.5)&(business_hours['th_s']<10.5)|
    (business_hours['fr_s']> 4.5)&(business_hours['fr_s']<10.5),1,0)

business_hours['weekend_breakfast']=np.where(
    (business_hours['sa_s']> 4.5)&(business_hours['sa_s']<10.5)|
    (business_hours['su_s']> 4.5)&(business_hours['su_s']<10.5),1,0)

business_hours['weekday_lunch']=np.where(
    (business_hours['mo_s']< 10.5)&(business_hours['mo_f']>15.5)|
    (business_hours['tu_s']< 10.5)&(business_hours['tu_f']>15.5)|
    (business_hours['we_s']< 10.5)&(business_hours['we_f']>15.5)|
    (business_hours['th_s']< 10.5)&(business_hours['th_f']>15.5)|
    (business_hours['fr_s']< 10.5)&(business_hours['fr_f']>15.5),1,0)

business_hours['weekend_lunch']=np.where(
    (business_hours['sa_s']< 11)&(business_hours['sa_f']>15.5)|
    (business_hours['su_s']< 11)&(business_hours['su_f']>15.5),1,0)

business_hours['weekday_dinner']=np.where(
    (business_hours['mo_s']< 15.5)&(business_hours['mo_f']>20)|
    (business_hours['tu_s']< 15.5)&(business_hours['tu_f']>20)|
    (business_hours['we_s']< 15.5)&(business_hours['we_f']>20)|
    (business_hours['th_s']< 15.5)&(business_hours['th_f']>20)|
    (business_hours['fr_s']< 15.5)&(business_hours['fr_f']>20),1,0)

business_hours['weekend_dinner']=np.where(
    (business_hours['sa_s']< 15.5)&(business_hours['sa_f']>20)|
    (business_hours['su_s']< 15.5)&(business_hours['su_f']>20),1,0)

business_hours['weekday_late_night']=np.where(
    (business_hours['mo_s']< 23.5)&(business_hours['mo_f']<4)|
    (business_hours['tu_s']< 23.5)&(business_hours['tu_f']<4)|
    (business_hours['we_s']< 23.5)&(business_hours['we_f']<4)|
    (business_hours['th_s']< 23.5)&(business_hours['th_f']<4)|
    (business_hours['fr_s']< 23.5)&(business_hours['fr_f']<4),1,0)

business_hours['weekend_late_night']=np.where(
    (business_hours['sa_s']< 23.5)&(business_hours['sa_f']<4)|
    (business_hours['su_s']< 23.5)&(business_hours['su_f']<4),1,0)

business_hours

,business_id,monday,tuesday,wednesday,thursday,friday,saturday,sunday,mo_s,mo_f,...,open_fridays,open_weekends,weekday_breakfast,weekend_breakfast,weekday_lunch,weekend_lunch,weekday_dinner,weekend_dinner,weekday_late_night,weekend_late_night
0,FYWN1wneV18bWNgQjJ2GNg,1,1,1,1,1,0,0,7.0,17.0,...,1,0,1,0,1,0,0,0,0,0
1,He-G7vWjzVUysIKrfNbPUQ,1,1,1,1,1,1,0,9.0,20.0,...,1,1,1,1,1,1,0,0,0,0
2,KQPW8lFf1y5BT2MxiSZ3QA,0,0,0,0,0,0,0,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0
3,8DShNS-LuFqpEWIp0HxijA,1,1,1,1,1,1,1,10.0,21.0,...,1,1,1,1,1,1,1,1,0,0
4,PfOCPjBrlQAnz__NXj9h_w,1,1,1,1,1,1,1,11.0,1.0,...,1,1,0,0,0,0,0,0,1,1
5,o9eMRCWt5PkpLDE0gOPtcQ,1,1,1,1,1,1,0,18.0,0.0,...,1,1,0,0,0,0,0,0,1,1
6,kCoE3jvEtg6UVz5SOD3GVw,1,1,1,1,1,0,0,8.0,17.0,...,1,0,1,0,1,0,0,0,0,0
7,OD2hnuuTJI9uotcKycxg1A,1,1,1,1,1,1,1,11.0,19.0,...,1,1,0,1,0,1,0,0,0,0
8,EsMcGiZaQuG1OOvL9iUFug,0,0,0,0,0,0,0,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0
9,TGWhGNusxyMaA4kQVBNeew,1,1,1,1,1,1,1,9.0,18.0,...,1,1,1,1,1,1,0,0,0,0


In [232]:
bus_atts=business_attributes.drop(['AcceptsInsurance','HairSpecializesIn_coloring','HairSpecializesIn_africanamerican',
                                   'HairSpecializesIn_curly','HairSpecializesIn_perms','HairSpecializesIn_kids',
                                   'HairSpecializesIn_extensions','HairSpecializesIn_asian',
                                   'HairSpecializesIn_straightperms'],axis=1)

In [233]:
dfs = [business, bus_atts, business_hours] # list of dataframes

df_merged = reduce(lambda  left,right: pd.merge(left,right,on=['business_id'],
                                            how='left'), dfs)
df_merged=df_merged.drop(['neighborhood'],axis=1)
print(df_merged.shape)
df_merged.head(50)

(3247, 132)


,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,...,open_fridays,open_weekends,weekday_breakfast,weekend_breakfast,weekday_lunch,weekend_lunch,weekday_dinner,weekend_dinner,weekday_late_night,weekend_late_night
0,__aKnGBedQ51_hEc3D9ARw,"""Paradise Bakery & Cafe""","""2502 E Camelback Rd, Ste 119""",Phoenix,AZ,85016,33.510270,-112.028937,3.0,75,...,1,1,1,1,1,1,1,1,0,0
1,_4bbY5joxS7ZM-027Gf6Jw,"""Mikado Sushi""","""3941 E Chandler Blvd, Ste 107""",Phoenix,AZ,85048,33.305108,-111.999400,4.5,7,...,0,0,0,0,0,0,0,0,0,0
2,_4n7K_UdRSEMw4vJdsU9jQ,"""Ray's Pizza""","""3414 W Union Hills Dr, Ste 5""",Phoenix,AZ,85027,33.655621,-112.132794,3.5,44,...,1,1,1,1,1,1,1,1,0,0
3,_6mvQX9x66oz046eS5ii8w,"""Juan's Authentic Mexican Food""","""1516 E Thomas Rd""",Phoenix,AZ,85014,33.480492,-112.049023,4.0,186,...,1,1,1,1,1,1,1,0,0,0
4,_70P6YlJVWHQq__6NpIWkA,"""Perc Up Coffee Cafe""","""2325 N 7th St""",Phoenix,AZ,85006,33.474169,-112.064757,4.0,14,...,1,1,1,1,1,1,0,0,0,0
5,_7hhpjQHxjPi0gNzzMmO6g,"""New Sun Star Super Buffet""","""5819 S Central Ave""",Phoenix,AZ,85040,33.393691,-112.073216,2.0,7,...,0,0,0,0,0,0,0,0,0,0
6,_7u6Cdgoo65xqUNOuRX4Ew,"""Nekter Juice Bar""","""4340 Indian School Rd, Ste 19""",Phoenix,AZ,85018,33.495424,-111.988167,4.0,69,...,1,1,1,1,1,1,0,0,0,0
7,_8AlKWBLY9F9zX8siTOr9A,"""Pita Pit""","""2110 W Southern, Ste B""",Mesa,AZ,85013,33.393657,-111.877179,4.5,19,...,1,1,0,0,0,0,0,0,0,0
8,_8c3Ky90nE50rN4Mb_Sxtw,"""Hsin Cafe""","""120 E Taylor St, Ste 150""",Phoenix,AZ,85004,33.453806,-112.071913,3.5,61,...,1,1,0,0,0,0,1,1,0,0
9,_8HzIslOdrED7JQF2DU2fg,"""Subway""","""8930 N 7th St""",Phoenix,AZ,85020,33.567147,-112.065592,3.0,14,...,0,0,0,0,0,0,0,0,0,0


In [234]:
df_merged.to_csv('C:/Users/Ric/Desktop/000_MY_WORKDIRECTORY/python/capstone yelp/business.csv',sep=',',index= False)

In [235]:
zipcodes = business.postal_code.unique().tolist()
len(zipcodes) # 41

41

In [236]:
demographics.count()
demographics.shape #(32977, 122)
demographics.head() #nulls might be non existant or they might be indicated with 0

# Create dataframe of demographics only with zip codes from business file
demographics_df=demographics[demographics.zcta.isin(zipcodes)]
demographics_df.shape #(41, 122) - matched all 41 zip codes

# Rename zip code column
demographics_df = demographics_df.rename(columns={'zcta': 'postal_code'})
demographics_df.count()

postal_code           41
primary_City          41
state                 41
primary_County        41
land_Area             41
water_Area            41
pop_2010              41
pop_2000              41
pop_Density           41
pop_Est_ACS           41
pop_Est_Factor        41
pop_Est_Level         41
poverty_Pct           41
group_Qtrs_Pct        41
college_Pct           41
degree_Pct            41
foreign_Pct           41
hH_Total              41
hH_Size               41
hH_Est                41
hH_Avg_Income         41
hH_Med_Income         41
hH_Earnings           41
hH_Earnings_Amt       41
hH_Retire             41
hH_Retire_Amt         41
hh_ss                 41
hH_SS_Amt             41
hh_ssi                41
hH_SSI_Amt            41
hH_Cash_Assist        41
hH_Cash_Assist_Amt    41
hH_Food_Stamps        41
family_Total          41
family_Size           41
family_Est            41
family_Avg_Income     41
family_Med_Income     41
per_Cap_Income        41
median_Age            41


In [237]:
business_df = pd.merge(df_merged,demographics_df,how='left',on='postal_code')

In [238]:
business_df

,business_id,name,address,city,state_x,postal_code,latitude,longitude,stars,review_count,...,hu_1970_1979,hu_1960_1969,hu_1950_1959,hu_1940_1949,hU_Before_1940,avg_Home_Value,med_Home_Value,avg_Rent,med_Rent,latLong
0,__aKnGBedQ51_hEc3D9ARw,"""Paradise Bakery & Cafe""","""2502 E Camelback Rd, Ste 119""",Phoenix,AZ,85016,33.510270,-112.028937,3.0,75,...,4094,3347,3861,1664,168,352964,247100,0,794,"33.50778, -112.03273100000001"
1,_4bbY5joxS7ZM-027Gf6Jw,"""Mikado Sushi""","""3941 E Chandler Blvd, Ste 107""",Phoenix,AZ,85048,33.305108,-111.999400,4.5,7,...,139,98,10,58,0,401034,344600,1242,1204,"33.302517, -112.02252800000001"
2,_4n7K_UdRSEMw4vJdsU9jQ,"""Ray's Pizza""","""3414 W Union Hills Dr, Ste 5""",Phoenix,AZ,85027,33.655621,-112.132794,3.5,44,...,3483,239,26,14,36,174196,161000,1000,966,"33.673176, -112.106301"
3,_6mvQX9x66oz046eS5ii8w,"""Juan's Authentic Mexican Food""","""1516 E Thomas Rd""",Phoenix,AZ,85014,33.480492,-112.049023,4.0,186,...,3021,2213,3329,1215,572,247225,209100,841,786,"33.510589, -112.058171"
4,_70P6YlJVWHQq__6NpIWkA,"""Perc Up Coffee Cafe""","""2325 N 7th St""",Phoenix,AZ,85006,33.474169,-112.064757,4.0,14,...,1152,1073,1593,2309,2264,173198,151600,0,696,"33.465234, -112.048771"
5,_7hhpjQHxjPi0gNzzMmO6g,"""New Sun Star Super Buffet""","""5819 S Central Ave""",Phoenix,AZ,85040,33.393691,-112.073216,2.0,7,...,2326,1010,1446,694,165,150615,122200,0,766,"33.40376, -112.022646"
6,_7u6Cdgoo65xqUNOuRX4Ew,"""Nekter Juice Bar""","""4340 Indian School Rd, Ste 19""",Phoenix,AZ,85018,33.495424,-111.988167,4.0,69,...,3787,3965,6865,543,404,477755,368200,0,761,"33.498821, -111.985541"
7,_8AlKWBLY9F9zX8siTOr9A,"""Pita Pit""","""2110 W Southern, Ste B""",Mesa,AZ,85013,33.393657,-111.877179,4.5,19,...,1484,1797,4050,1485,350,297034,223900,853,788,"33.507153, -112.084558"
8,_8c3Ky90nE50rN4Mb_Sxtw,"""Hsin Cafe""","""120 E Taylor St, Ste 150""",Phoenix,AZ,85004,33.453806,-112.071913,3.5,61,...,98,375,369,362,421,305711,264900,0,662,"33.450881, -112.070618"
9,_8HzIslOdrED7JQF2DU2fg,"""Subway""","""8930 N 7th St""",Phoenix,AZ,85020,33.567147,-112.065592,3.0,14,...,4200,2680,3404,570,141,310932,228100,0,867,"33.563095000000004, -112.05523500000001"


In [239]:
business_df.to_csv('C:/Users/Ric/Desktop/000_MY_WORKDIRECTORY/python/capstone yelp/business.csv',sep=',',index= False)